### Initial Setup

In [273]:
# importing necessary modules

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time

### Scraping information from the website to prepare a table containing only Arsenal fixtures

In [274]:
# web scraping from link containing the entire schedule of a season
link='https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
req=requests.get(link).text
html=BeautifulSoup(req,'html.parser')

# find table with the relevant id number
table=html.find('table', id='sched_2022-2023_9_1')

# assign all rows of the table to rows variable
rows=table.find_all('tr')

In [276]:
# assigning the col_data in a row, to a temporary dictionary
temp={}
for count in rows:
    data=count.find_all('td')
    temp[rows.index(count)]=data
temp.pop(0)

[]

In [277]:
# creating a final table list, to consolidate into a pandas dataframe
teams=[]
final_table=[]
for count in temp:
    matchday=temp[count]
    match_row=[]

    # adding the teams playing the first 10 matches, since each team will play for 
    # sure, considering its the first matchweek for each of the teams
    if len(teams)<20:
            teams.append(matchday[3].text)
            teams.append(matchday[7].text)

    # along with that, creating a table with only information we need, so removing details like,
    # number of attendees, venue name, refree name, etc
    for skip in matchday:
        if matchday.index(skip)>2 and matchday.index(skip)<8:
            match_row.append(skip.text)
        elif matchday.index(skip)==11:  
            link=skip.find('a')
            if link:
                href=link.get('href')
                match_row.append(href)
    final_table.append(match_row)
print(teams)

['Crystal Palace', 'Arsenal', 'Fulham', 'Liverpool', 'Tottenham', 'Southampton', 'Newcastle Utd', "Nott'ham Forest", 'Leeds United', 'Wolves', 'Bournemouth', 'Aston Villa', 'Everton', 'Chelsea', 'Leicester City', 'Brentford', 'Manchester Utd', 'Brighton', 'West Ham', 'Manchester City']


In [278]:
# final dataframe containing all matches and their results from the season
df=pd.DataFrame(final_table, columns=['home','hxg','score','axg','away', 'match_report'])
df.drop(index=10,inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,home,hxg,score,axg,away,match_report
0,Crystal Palace,1.2,0–2,1.0,Arsenal,/en/matches/e62f6e78/Crystal-Palace-Arsenal-Au...
1,Fulham,1.2,2–2,1.2,Liverpool,/en/matches/6713c1dc/Fulham-Liverpool-August-6...
2,Tottenham,1.5,4–1,0.5,Southampton,/en/matches/09d8a999/Tottenham-Hotspur-Southam...
3,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,/en/matches/1ac96eb4/Newcastle-United-Nottingh...
4,Leeds United,0.8,2–1,1.3,Wolves,/en/matches/82702941/Leeds-United-Wolverhampto...


In [279]:
# creating a dataframe containing only matches which arsenal have played
arsenal=df.loc[(df.home.str.contains('Arsenal',regex=True)) | (df.away.str.contains('Arsenal',regex=True))].reset_index(drop=True)
arsenal

,home,hxg,score,axg,away,match_report
0,Crystal Palace,1.2,0–2,1.0,Arsenal,/en/matches/e62f6e78/Crystal-Palace-Arsenal-Au...
1,Arsenal,2.7,4–2,0.5,Leicester City,/en/matches/7483b97f/Arsenal-Leicester-City-Au...
2,Bournemouth,0.3,0–3,1.3,Arsenal,/en/matches/7f11dd9e/Bournemouth-Arsenal-Augus...
3,Arsenal,2.6,2–1,0.8,Fulham,/en/matches/8b69fd2d/Arsenal-Fulham-August-27-...
4,Arsenal,2.4,2–1,0.4,Aston Villa,/en/matches/cc235aad/Arsenal-Aston-Villa-Augus...
5,Manchester Utd,1.5,3–1,1.3,Arsenal,/en/matches/61ddafa5/Manchester-United-Arsenal...
6,Brentford,0.5,0–3,1.5,Arsenal,/en/matches/fd5626e6/Brentford-Arsenal-Septemb...
7,Arsenal,2.4,3–1,1.6,Tottenham,/en/matches/aefe3b90/North-London-Derby-Arsena...
8,Arsenal,2.7,3–2,1.1,Liverpool,/en/matches/84a48413/Arsenal-Liverpool-October...
9,Leeds United,1.8,0–1,0.5,Arsenal,/en/matches/92885cfc/Leeds-United-Arsenal-Octo...


In [280]:
# mlink=arsenal.match_report.iloc[0]
# mlink='https://fbref.com'+mlink

In [281]:
# mreq=requests.get(mlink).text
# mhtml=BeautifulSoup(mreq,'html.parser')

### Just declaring all the variables that will later be the columns for the various stats at home and away games

In [282]:
apos=[]
hpos=[]
apa=[]
hpa=[]
asot=[]
hsot=[]

In [283]:
hcross,across,htouch,atouch,htackle,atackle,hint,aint,haer,aaer,hlb,alb=[],[],[],[],[],[],[],[],[],[],[],[]

In [284]:
htake_on,hsucc_take_ons,hlast_third_pass,hppa,hcrspa,hprog_pass,hcarry,hprog_carry,hlast_third_carry,hcpa,hprgr,hrecov=[],[],[],[],[],[],[],[],[],[],[],[]
atake_on,asucc_take_ons,alast_third_pass,appa,acrspa,aprog_pass,acarry,aprog_carry,alast_third_carry,acpa,aprgr,arecov=[],[],[],[],[],[],[],[],[],[],[],[]

In [285]:
# hcross,across,htouch,atouch,htackle,atackle,hint,aint,haer,aaer,hlb,alb

In [286]:
mr=arsenal.match_report
mr=mr.to_list()
condition=0
iterations_before_interval=20
delay_interval=(75)
iteration_count=0
for mlink in mr:
    iteration_count+=1
    pointer=mr.index(mlink)
    if arsenal.away.iloc[pointer]=='Arsenal':
        condition=1
    else:
        condition=0
    mlink='https://fbref.com'+mlink
    print(mlink)
    mreq=requests.get(mlink).text
    mhtml=BeautifulSoup(mreq,'html.parser')
    tables=mhtml.find_all('table')
    for count in tables:
        if count.find('span', class_='teamandlogo'):
            rows=count.find_all('tr')
            for skip in rows:
                if skip.text=='Possession':
                    index=rows.index(skip)
                    for data in rows[index+1]:
                        print(1,end=' ')
                        if condition==0 and rows[index+1].index(data)==1:
                            hpos.append(data.text.strip())
                        elif condition==1 and rows[index+1].index(data)==3:
                            apos.append(data.text.strip())
                elif skip.text=='Passing Accuracy':
                    index=rows.index(skip)
                    for data in rows[index+1]:
                        print(1,end=' ')
                        if condition==0 and rows[index+1].index(data)==1:
                                hpa.append(data.text.strip())
                        elif condition==1 and rows[index+1].index(data)==3:
                                apa.append(data.text.strip())
                elif skip.text=='Shots on Target':
                    index=rows.index(skip)
                    for data in rows[index+1]:
                        print(1,end=' ')
                        if condition==0 and rows[index+1].index(data)==1:
                            hsot.append(data.text.strip())
                        elif condition==1 and rows[index+1].index(data)==3:
                            asot.append(data.text.strip())
                print(iteration_count)
    divs=mhtml.find('div',id='team_stats_extra')
    huh=0
    wanted=['Crosses','Touches','Tackles','Interceptions','Aerials Won','Long Balls']
    for count in divs:
        huh+=1
        temp=[]
        for skip in count:
            if not huh%2:
                for jump in skip:
                    if jump:
                        temp.append(jump.strip())
        if temp:
            temp=temp[5:]
            print(temp)
            for count in wanted:
                if count in temp and not condition:
                    index=temp.index(count)
                    if count=='Crosses':
                        hcross.append(temp[index-1])
                    elif count=='Touches':
                        htouch.append(temp[index-1])
                    elif count=='Tackles':
                        htackle.append(temp[index-1])
                    elif count=='Interceptions':
                        hint.append(temp[index-1])
                    elif count=='Aerials Won':
                        haer.append(temp[index-1])
                    elif count=='Long Balls':
                        hlb.append(temp[index-1])
                    # print(temp[index-1])
                elif count in temp and condition:
                    index=temp.index(count)
                    if count=='Crosses':
                        across.append(temp[index-1])
                    elif count=='Touches':
                        atouch.append(temp[index-1])
                    elif count=='Tackles':
                        atackle.append(temp[index-1])
                    elif count=='Interceptions':
                        aint.append(temp[index-1])
                    elif count=='Aerials Won':
                        aaer.append(temp[index-1])
                    elif count=='Long Balls':
                        alb.append(temp[index-1])


    summary_table=mhtml.find(lambda tag: tag.name=='table' and 'stats_18bb7c10_summary' in tag.get('id',''))
    footer=summary_table.find('tfoot')
    
    take_on=footer.find('td',{'data-stat':'take_ons'}).text
    if not condition:
        htake_on.append(take_on)
    elif condition:
        atake_on.append(take_on)
    
    succ_take_ons=footer.find('td',{'data-stat':'take_ons_won'}).text
    if not condition:
        hsucc_take_ons.append(succ_take_ons)
    elif condition:
        asucc_take_ons.append(succ_take_ons)



    passing_table=mhtml.find(lambda tag: tag.name=='table' and 'stats_18bb7c10_passing' in tag.get('id',''))
    footer=passing_table.find('tfoot')
    
    last_third_pass=footer.find('td',{'data-stat':'passes_into_final_third'}).text
    if not condition:
        hlast_third_pass.append(last_third_pass)
    elif condition:
        alast_third_pass.append(last_third_pass)

    ppa=footer.find('td',{'data-stat':'passes_into_penalty_area'}).text
    if not condition:
        hppa.append(ppa)
    elif condition:
        appa.append(ppa)

    crspa=footer.find('td',{'data-stat':'crosses_into_penalty_area'}).text
    if not condition:
        hcrspa.append(crspa)
    elif condition:
        acrspa.append(crspa)

    prog_pass=footer.find('td',{'data-stat':'progressive_passes'}).text
    if not condition:
        hprog_pass.append(prog_pass)
    elif condition:
        aprog_pass.append(prog_pass)



    possession_table=mhtml.find(lambda tag: tag.name=='table' and 'stats_18bb7c10_possession' in tag.get('id',''))
    footer=possession_table.find('tfoot')

    prog_carry=footer.find('td',{'data-stat':'progressive_carries'}).text
    if not condition:
        hprog_carry.append(prog_carry)
    elif condition:
        aprog_carry.append(prog_carry)

    carry=footer.find('td',{'data-stat':'carries'}).text
    if not condition:
        hcarry.append(carry)
    elif condition:
        acarry.append(carry)

    last_third_carry=footer.find('td',{'data-stat':'carries_into_final_third'}).text
    if not condition:
        hlast_third_carry.append(last_third_carry)
    elif condition:
        alast_third_carry.append(last_third_carry)
    
    cpa=footer.find('td',{'data-stat':'carries_into_penalty_area'}).text
    if not condition:
        hcpa.append(cpa)
    elif condition:
        acpa.append(cpa)

    prgr=footer.find('td',{'data-stat':'progressive_passes_received'}).text
    if not condition:
        hprgr.append(prgr)
    elif condition:
        aprgr.append(prgr)



    miscellaneuos_table=mhtml.find(lambda tag: tag.name=='table' and 'stats_18bb7c10_misc' in tag.get('id',''))
    footer=miscellaneuos_table.find('tfoot')

    recov=footer.find('td',{'data-stat':'ball_recoveries'}).text
    if not condition:
        hrecov.append(recov)
    elif condition:
        arecov.append(recov)

    
    
    if iteration_count%iterations_before_interval==0:
        time.sleep(delay_interval)

https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League
1
1 1 1 1 1 1
1
1 1 1 1 1 1
1
1 1 1 1 1 1
1
1
1
1
1
['16', 'Fouls', '11', '', '3', 'Corners', '5', '', '19', 'Crosses', '11', '', '726', 'Touches', '599', '']
['18', 'Tackles', '29', '', '8', 'Interceptions', '9', '', '10', 'Aerials Won', '14', '', '18', 'Clearances', '24', '']
['1', 'Offsides', '2', '', '4', 'Goal Kicks', '2', '', '25', 'Throw Ins', '14', '', '83', 'Long Balls', '59', '']
https://fbref.com/en/matches/7483b97f/Arsenal-Leicester-City-August-13-2022-Premier-League
2
1 1 1 1 1 2
2
1 1 1 1 1 2
2
1 1 1 1 1 2
2
2
2
2
2
['15', 'Fouls', '9', '', '6', 'Corners', '2', '', '17', 'Crosses', '9', '', '590', 'Touches', '586', '']
['8', 'Tackles', '12', '', '8', 'Interceptions', '8', '', '19', 'Aerials Won', '14', '', '12', 'Clearances', '19', '']
['2', 'Offsides', '3', '', '3', 'Goal Kicks', '12', '', '15', 'Throw Ins', '16', '', '50', 'Long Balls', '66', '']
https://fbref.com/en/matches/7f11dd

In [287]:
for count in range(len(hpa)):
    hpos[count]=hpos[count][:2]
    apos[count]=apos[count][:2]
    apa[count]=apa[count].split()
    apa[count]=apa[count][2::2]
    hpa[count]=hpa[count].split()
    hpa[count]=hpa[count][:3:2]
    asot[count]=asot[count].split()
    asot[count]=asot[count][2::2]
    hsot[count]=hsot[count].split()
    hsot[count]=hsot[count][:3:2]

In [288]:
hpos=hpos[:19]
apos=apos[:19]

In [289]:
hpa_com=[]
hpa_att=[]
apa_com=[]
apa_att=[]
hsot_com=[]
hsot_att=[]
asot_com=[]
asot_att=[]
for count in range(len(hpa)):
    hpa_com.append(hpa[count][0])
    hpa_att.append(hpa[count][1])
    apa_com.append(apa[count][0])
    apa_att.append(apa[count][1])
    hsot_com.append(hsot[count][0])
    hsot_att.append(hsot[count][1])
    asot_com.append(asot[count][0])
    asot_att.append(asot[count][1])
print(hpos,apos,hpa_com,hpa_att,apa_com,apa_att,hsot_com,hsot_att,asot_com,asot_att)

['50', '71', '59', '64', '43', '69', '66', '66', '57', '69', '63', '73', '79', '62', '66', '74', '55', '41', '51'] ['44', '57', '60', '63', '53', '59', '55', '62', '32', '49', '70', '67', '65', '55', '41', '70', '48', '45', '81'] ['407', '534', '452', '539', '316', '580', '606', '480', '437', '537', '466', '681', '675', '528', '564', '611', '485', '292', '418'] ['499', '637', '549', '613', '417', '667', '699', '572', '555', '649', '569', '794', '801', '629', '668', '729', '577', '387', '484'] ['378', '534', '418', '525', '437', '494', '388', '537', '225', '408', '584', '530', '524', '487', '301', '547', '400', '291', '665'] ['469', '614', '508', '624', '534', '582', '494', '622', '310', '494', '687', '643', '654', '578', '399', '643', '481', '374', '764'] ['7', '8', '8', '9', '6', '9', '5', '4', '5', '7', '0', '5', '9', '5', '4', '5', '9', '2', '8'] ['19', '22', '22', '22', '10', '24', '16', '17', '25', '23', '9', '15', '31', '15', '12', '25', '16', '14', '14'] ['2', '6', '3', '7', '4'

In [290]:
home23_data=[hpos,hpa_com,hpa_att,hsot_com,hsot_att,hcross,htouch,htackle,hint,haer,hlb,htake_on,hsucc_take_ons,hlast_third_pass,hppa,hcrspa,hprog_pass,hcarry,hprog_carry,hlast_third_carry,hcpa,hprgr,hrecov]
away23_data=[apos,apa_com,apa_att,asot_com,asot_att,across,atouch,atackle,aint,aaer,alb,atake_on,asucc_take_ons,alast_third_pass,appa,acrspa,aprog_pass,acarry,aprog_carry,alast_third_carry,acpa,aprgr,arecov]

In [291]:
home23=pd.DataFrame(home23_data).transpose()
home23.columns=['possession','passes_completed','passes_attempted','shots_on_target','shots_total','crosses','touches','tackles','interceptions','aerial_duels','long_balls','take_ons','take_ons_won','1/3_pass','pass_in_pen','cross_in_pen','prog_pass','carry','prog_carry','1/3_carry','carry_in_pen','prog_pass_rec','recoveries']
away23=pd.DataFrame(away23_data).transpose()
away23.columns=['possession','passes_completed','passes_attempted','shots_on_target','shots_total','crosses','touches','tackles','interceptions','aerial_duels','long_balls','take_ons','take_ons_won','1/3_pass','pass_in_pen','cross_in_pen','prog_pass','carry','prog_carry','1/3_carry','carry_in_pen','prog_pass_rec','recoveries']

In [299]:
away23

,possession,passes_completed,passes_attempted,shots_on_target,shots_total,crosses,touches,tackles,interceptions,aerial_duels,...,1/3_pass,pass_in_pen,cross_in_pen,prog_pass,carry,prog_carry,1/3_carry,carry_in_pen,prog_pass_rec,recoveries
0,44,378,469,2,10,19,726,18,8,10,...,34,10,0,40,359,15,8,5,39,55
1,57,534,614,6,14,13,546,22,7,12,...,24,6,2,35,526,14,19,3,35,56
2,60,418,508,3,16,6,452,17,7,10,...,38,19,3,58,311,22,10,7,58,47
3,63,525,624,7,13,18,463,21,12,21,...,47,7,0,41,484,13,15,2,41,53
4,53,437,534,4,9,19,578,25,13,13,...,21,2,0,42,365,16,11,4,41,57
5,59,494,582,3,12,20,508,21,3,11,...,62,8,2,61,428,15,5,7,61,56
6,55,388,494,2,14,15,508,24,9,17,...,38,7,1,48,308,25,9,9,46,69
7,62,537,622,3,14,15,476,17,6,20,...,66,11,2,49,486,20,16,5,49,47
8,32,225,310,7,14,24,738,8,9,12,...,17,7,0,29,223,16,8,9,29,44
9,49,408,494,5,14,22,621,17,11,21,...,49,6,1,48,382,27,16,8,46,49


In [300]:
home23

,possession,passes_completed,passes_attempted,shots_on_target,shots_total,crosses,touches,tackles,interceptions,aerial_duels,...,1/3_pass,pass_in_pen,cross_in_pen,prog_pass,carry,prog_carry,1/3_carry,carry_in_pen,prog_pass_rec,recoveries
0,50,407,499,7,19,17,590,8,8,19,...,29,7,2,29,323,23,19,9,29,50
1,71,534,637,8,22,19,745,15,7,11,...,42,17,3,66,445,17,18,8,63,53
2,59,452,549,8,22,20,665,10,8,10,...,40,16,0,50,420,23,19,7,50,50
3,64,539,613,9,22,25,715,14,6,9,...,68,16,7,78,504,23,8,10,77,50
4,43,316,417,6,10,9,542,16,6,13,...,37,11,0,42,277,17,11,7,41,43
5,69,580,667,9,24,23,763,14,7,10,...,41,18,3,58,427,38,21,12,55,51
6,66,606,699,5,16,25,795,15,4,15,...,46,19,1,78,523,23,18,4,78,61
7,66,480,572,4,17,16,675,13,3,12,...,53,10,1,56,378,21,11,9,54,48
8,57,437,555,5,25,22,677,21,9,11,...,34,10,1,49,345,37,21,18,49,64
9,69,537,649,7,23,36,745,10,4,13,...,48,26,4,89,477,32,24,13,89,45


In [294]:
home23.columns

Index(['possession', 'passes_completed', 'passes_attempted', 'shots_on_target',
       'shots_total', 'crosses', 'touches', 'tackles', 'interceptions',
       'aerial_duels', 'long_balls', 'take_ons', 'take_ons_won', '1/3_pass',
       'pass_in_pen', 'cross_in_pen', 'prog_pass', 'carry', 'prog_carry',
       '1/3_carry', 'carry_in_pen', 'prog_pass_rec', 'recoveries'],
      dtype='object')